In [105]:
import numpy as np
import pandas as pd
from osgeo import gdal
from gdalconst import *
#import osr
import matplotlib.pyplot as plt
%matplotlib qt




In [106]:
#Read a raster
f = gdal.Open("/Users/bmelosh/VagrantBoxes/LSDTopoTools/Topographic_projects/Test_data/Mandakini_HS.bil")
fA = f.ReadAsArray()

#Read stream data
name = "Mandakini_fullProfileMC_forced_0.45_20_2_20_90_281_for_Arc.csv" #Set file name
path = "/Users/bmelosh/VagrantBoxes/LSDTopoTools/Topographic_projects/Test_data/deeper/" #Set path to data
#for csv files
data = np.genfromtxt((path+name), delimiter=',', skip_header=1, names=['id', 'x', 'y', 'chan_number', 'reciever_chan','node_on_reciever_chan', 'node', 'row','column', 'flow_distance', 'chi', 'elevation', 'drainage_area', 'n_data_points', 'm_mean','m_st_dev', 'm_std_err', 'b_mean', 'b_st_dev', 'b_std_err', 'DW_mean', 'DW_st_dev', 'DW_std_err', 'fitted_elev_mean', 'fitted_elev_stdev', 'fitted_elev_std_err'])
#for tree files
#data = np.genfromtxt((path+name), delimiter=' ', skip_header=1, names=['chan_number', 'reciever_chan','node_on_reciever_chan', 'node', 'row','column', 'flow_distance', 'chi', 'elevation', 'drainage_area', 'n_data_points', 'm_mean','m_st_dev', 'm_std_err', 'b_mean', 'b_st_dev', 'b_std_err', 'DW_mean', 'DW_st_dev', 'DW_std_err', 'fitted_elev_mean', 'fitted_elev_stdev', 'fitted_elev_std_err'])


In [107]:
Test_data = ("/Users/bmelosh/VagrantBoxes/LSDTopoTools/Topographic_projects/Test_data/")
Test = gdal.Open(Test_data + "/Mandakini.bil")
M_HS = Test.ReadAsArray()#.astype(np.float)


def plot_coord(Test):

    (upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = Test.GetGeoTransform()
    dx = x_size
    dy = y_size
    nx, ny = Test.RasterXSize, Test.RasterYSize  # Size of the original raster

    xllcenter = upper_left_x + dx/2  # x coordinate center of lower left pxl
    yllcenter = upper_left_y - dx/2 # y coordinate center of lower left pxl   - (ny-1)*dx 

    #Create arrays of the x and y coordinates of each pixel (the axes)
    xcoordinates = [x*dx + xllcenter for x in range(nx)]
    ycoordinates = [y*dy + yllcenter for y in range(ny)]

    #Create 2 2d grids describing x and y coordinates
    X,Y = np.meshgrid(xcoordinates, ycoordinates) 

    fig = plt.contourf(X, Y, M_HS, levels=np.linspace(np.amin(M_HS[M_HS > 0]),np.amax(M_HS), 50))
    
    return fig



In [108]:

cm = plt.cm.gist_earth


fig, ax = plt.subplots(2, figsize=(10,10), sharex=False) #define 2 subfigures

ax[0].contourf(X, Y, M_HS, levels=np.linspace(np.amin(M_HS[M_HS > 0]),np.amax(M_HS), 50))
ax[0].plot(data['x'], data['y'], 'ro')
#ax[0].imshow(fA, interpolation='nearest', vmin=0, cmap=plt.cm.gray) 
ax[0].set_aspect('equal')
ax[0].set_title('Terrain map with stream channel')
ax[0].set_ylabel('Northing')
ax[0].set_xlabel('Easting')



ax[1].scatter(data['x'], data['y'])
ax[1].set_title('Stream Map')
ax[1].set_ylabel('Northing')
ax[1].set_xlabel('Easting')

plt.show()

In [4]:
#gdalinfo "/Users/bmelosh/VagrantBoxes/LSDTopoTools/Topographic_projects/Test_data/Mandakini_HS.bil"